In [1]:
import pandas as pd  
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from sqlalchemy import create_engine, inspect, text
import pyodbc
from tqdm import tqdm
from typing import List
import sqlalchemy
from scipy.stats import norm
import time
import re
import joblib
import numpy as np
import os
from sklearn.model_selection import KFold
from scipy.stats import mode
from scipy.stats import gaussian_kde
import statsmodels.api as sm
from sklearn.mixture import GaussianMixture
from scipy.stats import norm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier, 
                             ExtraTreesClassifier, AdaBoostClassifier)
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("full_specnr_joined.csv")
df

,SPECNR,MANDT,SEQNR,DISKNR,SPECTYPE,CONMETHOD,TIK,TIKP,TIKM,DEFUALTTIK,...,TARGETDSTYRIGHT,MASKAR,MASKARC,EXCTIK,JIG_CHK,KUNNR,SPART,MACTIVE,KDMAT,REV
0,21CSP002-04,300.0,1.0,1000050281,BUM-B,MSAP,0.25,0.02,0.02,NaN,...,0.0,40.0,0.0,0.0,NaN,10005.0,30.0,X,ADS92641,000
1,21CSP002-05,300.0,2.0,1000050281,BUM-B,MSAP,0.25,0.02,0.02,NaN,...,0.0,0.0,0.0,0.0,NaN,10005.0,30.0,NaN,ADS92641,000
2,21CSP002-06,300.0,1.0,1000050281,BUM-B,MSAP,0.25,0.02,0.02,NaN,...,0.0,0.0,0.0,0.0,NaN,10005.0,30.0,NaN,ADS92641,000
3,21CSP002-07,300.0,2.0,1000050281,BUM-B,MSAP,0.25,0.02,0.02,NaN,...,0.0,0.0,0.0,0.0,NaN,10005.0,30.0,NaN,ADS92641,000
4,21CSP002-08,300.0,2.0,1000050281,BUM-B,MSAP,0.25,0.02,0.02,NaN,...,0.0,0.0,0.0,0.0,NaN,10005.0,30.0,NaN,ADS92641,000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9160,25SIP015-02,300.0,3.0,1000058039,NORMAL,TENTING,0.25,0.04,0.04,NaN,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
9161,25SIP016-01,300.0,2.0,1000058047,NORMAL,TENTING,0.25,0.05,0.05,NaN,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
9162,25SIP017-01,300.0,2.0,1000058332,NORMAL,TENTING,0.25,0.04,0.04,NaN,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
9163,25SIP018-01,300.0,2.0,1000058022,NORMAL,TENTING,0.25,0.04,0.04,NaN,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df = pd.read_csv("full_specnr_joined_1102.csv")
df

,SPECNR,MANDT,SEQNR,DISKNR,SPECTYPE,CONMETHOD,TIK,TIKP,TIKM,DEFUALTTIK,...,FLEXCH,AUTOPUNCH,TARGETDSTXTOP,TARGETDSTXBOT,TARGETDSTYLEFT,TARGETDSTYRIGHT,MASKAR,MASKARC,EXCTIK,JIG_CHK
0,21CSP002-04,300,1,1000050281,BUM-B,MSAP,0.25,0.02,0.02,NaN,...,0,0,0,0,0.0,0.0,40.0,0.0,0.0,NaN
1,21CSP002-05,300,2,1000050281,BUM-B,MSAP,0.25,0.02,0.02,NaN,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN
2,21CSP002-06,300,1,1000050281,BUM-B,MSAP,0.25,0.02,0.02,NaN,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN
3,21CSP002-07,300,2,1000050281,BUM-B,MSAP,0.25,0.02,0.02,NaN,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN
4,21CSP002-08,300,2,1000050281,BUM-B,MSAP,0.25,0.02,0.02,NaN,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9160,25SIP015-02,300,3,1000058039,NORMAL,TENTING,0.25,0.04,0.04,NaN,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN
9161,25SIP016-01,300,2,1000058047,NORMAL,TENTING,0.25,0.05,0.05,NaN,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN
9162,25SIP017-01,300,2,1000058332,NORMAL,TENTING,0.25,0.04,0.04,NaN,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN
9163,25SIP018-01,300,2,1000058022,NORMAL,TENTING,0.25,0.04,0.04,NaN,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN


In [24]:
df['MATNR'].value_counts()

MATNR
2000325    2662
2001023     549
1008017     402
2002595     341
2002653     297
           ... 
1020840       1
1020868       1
1019885       1
1007889       1
1008908       1
Name: count, Length: 484, dtype: int64

### 수치형

In [5]:
CONFIG = {
    'test_size': 0.2,
    'random_state': 42,
    'max_categorical_unique': 100,
    'cv_folds': 3,
    'n_top_features': 20,
    'figsize_confusion': (8, 6),
    'figsize_importance': (12, 8),
    'weight_threshold': 0.05,
    'n_components_range': range(1, 11)
}

models = {
    'RandomForest': RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=CONFIG['random_state']
    ),
    'XGBoost': XGBClassifier(
        n_estimators=100,
        max_depth=5,
        learning_rate=0.1,
        random_state=CONFIG['random_state'],
        verbosity=0,
        base_score=0.5
    ),
    'DecisionTree': DecisionTreeClassifier(
        max_depth=10,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=CONFIG['random_state']
    )
}

In [6]:
def safe_stratified_split(X, y, test_size=0.2, random_state=42):
    class_counts = y.value_counts()
    min_class_count = class_counts.min()
    n_classes = len(class_counts)

    test_size_absolute = int(test_size * len(y)) if isinstance(test_size, float) else test_size

    if min_class_count >= 2 and test_size_absolute >= n_classes:
        return train_test_split(
            X, y, stratify=y, test_size=test_size, random_state=random_state
        ), True
    else:
        print(f" 데이터 품질문제: 최소 클래스 샘플 수={min_class_count}, "
              f"클래스 수={n_classes}, test_size={test_size_absolute}")
        return train_test_split(
            X, y, test_size=test_size, random_state=random_state
        ), False

def normalize_binary_labels(y):
    unique_classes = np.unique(y)
    if len(unique_classes) == 2:
        if set(unique_classes) != {0, 1}:
            mapping = {unique_classes[0]: 0, unique_classes[1]: 1}
            y = y.map(mapping)
    return y


def prepare_data_for_boosting(X_data):
    if isinstance(X_data, pd.DataFrame):
        return X_data.values.astype(np.float32)
    return X_data.astype(np.float32)

def train_and_predict(model_name, model, X_train, X_test, y_train):
    try:
        if model_name == 'XGBoost':
            import xgboost as xgb
            model_params = model.get_params()
            n_classes = y_train.nunique()

            if n_classes > 2:
                model_params.update({'objective': 'multi:softprob', 'num_class': n_classes})
            elif n_classes == 2:
                model_params.update({'objective': 'binary:logistic'})
            else:
                print(f"[{model_name}] 클래스가 하나! 학습 불가능")
                return None, None

            model = xgb.XGBClassifier(**model_params)
            X_train_proc = prepare_data_for_boosting(X_train)
            X_test_proc = prepare_data_for_boosting(X_test)

        else:
            X_train_proc, X_test_proc = X_train, X_test

        model.fit(X_train_proc, y_train)
        y_pred = model.predict(X_test_proc)

        return y_pred, model

    except Exception as e:
        print(f"[{model_name}] 모델 훈련 중 오류: {str(e)}")
        return None, None
        
def cluster_and_classify(variable, df, models, config, weight_threshold=0.05, n_components_range=range(1, 11)):
    # 클러스터링할 데이터 선택
    df_var = df[df[variable].notna() & (df[variable] != 0)].copy()
    y_raw = df_var[variable].dropna().values.reshape(-1, 1)
    
    # 컴포넌트 수 설정
    max_n_components = min(len(y_raw), config['n_components_range'].stop - 1)
    if max_n_components < 2:
        print(f"[{variable}] 유효한 클러스터 수 없음 (샘플 수={len(y_raw)}), 스킵")
        return None
    
    # GMM 클러스터링 
    lowest_bic = np.inf
    best_gmm = None
    for n_components in range(1, max_n_components + 1):
        gmm = GaussianMixture(n_components=n_components, random_state=42)
        gmm.fit(y_raw)
        bic = gmm.bic(y_raw)
        if bic < lowest_bic:
            lowest_bic = bic
            best_gmm = gmm

    if best_gmm is None:
        print(f"[{variable}] GMM fitting 실패")
        return None

    initial_labels = best_gmm.predict(y_raw)
    weights = best_gmm.weights_
    significant_clusters = [i for i, w in enumerate(weights) if w >= weight_threshold]
    filtered_data = y_raw[np.isin(initial_labels, significant_clusters)]

    final_gmm = GaussianMixture(n_components=len(significant_clusters), random_state=42)
    final_gmm.fit(filtered_data)
    filtered_labels = final_gmm.predict(filtered_data)

    # 우세 클러스터만 필터링
    selected_indices = df_var.index[np.isin(initial_labels, significant_clusters)]
    df_filtered = df.loc[selected_indices].copy()
    df_filtered['final_gmm_label'] = filtered_labels

    # 타겟 변수
    y_target = df_filtered['final_gmm_label']
    y_target = normalize_binary_labels(y_target)

    if y_target.nunique() < 2:
        print(f"[{variable}] 클러스터 클래스가 하나! 학습 건너뜀.")
        return None

    # 피처 추출 
    exclude_cols = [variable, 'final_gmm_label']
    feature_cols = [col for col in df_filtered.columns if col not in exclude_cols]

    X_num = df_filtered[feature_cols].select_dtypes(include=[np.number]).fillna(0)
    X_cat = df_filtered[feature_cols].select_dtypes(exclude=[np.number]).fillna('missing')

    # 범주형 인코딩
    X_cat_enc = pd.DataFrame(index=X_cat.index)
    for col in X_cat.columns:
        le = LabelEncoder()
        X_cat_enc[col] = le.fit_transform(X_cat[col].astype(str))
    X_cat_enc = X_cat_enc.astype(np.float32)

    # 수치형 스케일링
    scaler = StandardScaler()
    X_num_scaled = pd.DataFrame(scaler.fit_transform(X_num), columns=X_num.columns, index=X_num.index)

    # 최종 피처 결합 및 결측치/무한치 처리
    X_final = pd.concat([X_num_scaled, X_cat_enc], axis=1)
    X_final = X_final.replace([np.inf, -np.inf], np.nan).fillna(0)
    for col in X_final.columns:
        X_final[col] = pd.to_numeric(X_final[col], errors='coerce').fillna(0).astype(np.float32)

    # 8. train/test split
    (X_train, X_test, y_train, y_test), stratified = safe_stratified_split(
        X_final, y_target,
        test_size=config['test_size'],
        random_state=config['random_state']
    )

    from sklearn.metrics import accuracy_score, f1_score

    # 모델 학습, 평가
    feature_results = {}
    for model_name, model in models.items():
        y_pred, _ = train_and_predict(model_name, model, X_train, X_test, y_train)
        if y_pred is not None:
            # y_pred가 확률 배열이면 argmax로 변환
            if len(y_pred.shape) > 1 and y_pred.shape[1] > 1:
                y_pred = np.argmax(y_pred, axis=1)
            acc = accuracy_score(y_test, y_pred) * 100
            f1 = f1_score(y_test, y_pred, average='weighted') * 100  
            feature_results[model_name] = (acc, f1)
        else:
            feature_results[model_name] = (None, None)

    # 가장 좋은 모델과
    best_model = max(feature_results, key=lambda k: feature_results[k][0] if feature_results[k][0] is not None else -1)
    best_acc, best_f1 = feature_results[best_model]

    return variable, best_model, best_acc, best_f1

# 변수 리스트 
variable_list = ['SEQNR', 'TIK', 'TIKP', 'TIKM', 'SMTIK', 'TIKRATE', 'DVPDATE', 'TARTIK', 'TIKDIFF', 'BALLBOT', 'BALLTOP', 'UNITSIZEX', 'UNITSIZEY', 'UNITPITCH', 'UNITPITCHY', 'SPLXOUTRATE', 'SPLXOUT', 'XOUTRATE', 'XOUT', 'BTBX', 'TOLERANCEXM', 'TOLERANCEXP', 'BTBY', 'TOLERANCEYM', 'TOLERANCEYP', 'STRIPX', 'STRIPY', 'UPNR', 'UPCOL', 'UPROW', 'PANELLOSSX', 'PANELLOSSY', 'PCPPANEL', 'ARNR', 'ARM2NR', 'PLATEYIELD', 'STPDAT', 'OKDAT', 'OKZET', 'TOOLSERIAL', 'TOOLREV', 'REQNR', 'ERDAT', 'ERZET', 'AEDAT', 'AEZET', 'BOMNR', 'LAYER', 'MATNR', 'MATNRTIK', 'MATNRTIKT', 'COPPERTIKT', 'TOLERANCEP', 'TOLERANCEM', 'LABST', 'LABSTF', 'POSNR', 'LAYERTO', 'TIKAL', 'TOLERANCEPM', 'PROCSEQ', 'PRLAYERTO', 'SVH', 'SVHPAD', 'SVHAR', 'ZSUM', 'CAPTUREPAD', 'TOP1', 'BOTTOM1', 'LAYERPROC', 'LINEWID', 'WIDMN', 'WIDMX', 'LINESPACE', 'SPACEMN', 'SPACEMX', 'NWTAR', 'NWMIN', 'NWMAX', 'NSTAR', 'NSMIN', 'NSMAX', 'THTAR', 'THMIN', 'LINEWIDREF', 'CURATE', 'PADSEQ', 'PITCH', 'SEQ', 'WID', 'SPACEZ', 'OHMTOLPM', 'MASWID', 'MASSPACE', 'MASGND', 'AWWID', 'AWSPACE', 'AWGND', 'HOLESIZE', 'DHS', 'HOLENR', 'PROCGR', 'PROCSEQC', 'MASKAR', 'MASKARC', 'EXCTIK']

# 결과 저장
all_results = []
for var in variable_list:
    res = cluster_and_classify(var, df, models, CONFIG)
    if res is not None:
        all_results.append(res)

print(len(variable_list))
print(len(all_results))
      
# 출력
for variable, model_name, acc, f1 in all_results:
    print(f"{variable} : {model_name} | Accuracy: {acc:.1f}%, F1-score: {f1:.1f}%")

[SPACEZ] 클러스터 클래스가 하나! 학습 건너뜀.
 데이터 품질문제: 최소 클래스 샘플 수=1, 클래스 수=5, test_size=19
106
105
SEQNR : RandomForest | Accuracy: 89.0%, F1-score: 85.4%
TIK : XGBoost | Accuracy: 99.9%, F1-score: 99.9%
TIKP : XGBoost | Accuracy: 99.9%, F1-score: 99.9%
TIKM : XGBoost | Accuracy: 100.0%, F1-score: 100.0%
SMTIK : XGBoost | Accuracy: 100.0%, F1-score: 100.0%
TIKRATE : XGBoost | Accuracy: 97.7%, F1-score: 97.7%
DVPDATE : XGBoost | Accuracy: 95.4%, F1-score: 95.5%
TARTIK : DecisionTree | Accuracy: 99.8%, F1-score: 99.8%
TIKDIFF : XGBoost | Accuracy: 96.5%, F1-score: 96.5%
BALLBOT : XGBoost | Accuracy: 98.0%, F1-score: 98.0%
BALLTOP : RandomForest | Accuracy: 96.2%, F1-score: 96.5%
UNITSIZEX : XGBoost | Accuracy: 99.5%, F1-score: 99.5%
UNITSIZEY : XGBoost | Accuracy: 99.5%, F1-score: 99.5%
UNITPITCH : DecisionTree | Accuracy: 100.0%, F1-score: 100.0%
UNITPITCHY : DecisionTree | Accuracy: 99.9%, F1-score: 99.9%
SPLXOUTRATE : XGBoost | Accuracy: 99.7%, F1-score: 99.7%
SPLXOUT : XGBoost | Accuracy: 99.8%,

In [7]:
# F1-score 기준
f1_100_vars = [r[0] for r in all_results if r[3] == 100]
f1_95_100_vars = [r[0] for r in all_results if r[3] is not None and 95 <= r[3] < 100]
f1_90_100_vars = [r[0] for r in all_results if r[3] is not None and 90 <= r[3] < 100]
f1_85_100_vars = [r[0] for r in all_results if r[3] is not None and 85 <= r[3] < 100]
f1_80_100_vars = [r[0] for r in all_results if r[3] is not None and 80 <= r[3] < 100]
f1_below_95_vars = [r[0] for r in all_results if r[3] is not None and r[3] < 95]
f1_below_90_vars = [r[0] for r in all_results if r[3] is not None and r[3] < 90]
f1_below_85_vars = [r[0] for r in all_results if r[3] is not None and r[3] < 85]
f1_below_80_vars = [r[0] for r in all_results if r[3] is not None and r[3] < 80]

print("F1-score가 정확히 100%인 변수 개수:", len(f1_100_vars))
print("F1-score가 95% 이상 100% 미만인 변수 개수:", len(f1_95_100_vars))
print("F1-score가 90% 이상 100% 미만인 변수 개수:", len(f1_90_100_vars))
print("F1-score가 85% 이상 100% 미만인 변수 개수:", len(f1_85_100_vars))
print("F1-score가 80% 이상 100% 미만인 변수 개수:", len(f1_80_100_vars))
print("F1-score가 95% 미만인 변수 개수:", len(f1_below_95_vars))
print("F1-score가 90% 미만인 변수 개수:", len(f1_below_90_vars))
print("F1-score가 85% 미만인 변수 개수:", len(f1_below_85_vars))
print("F1-score가 80% 미만인 변수 개수:", len(f1_below_80_vars))

print("\n F1 = 100%인 변수 목록:")
print(f1_100_vars)

print("\n F1 = 95% 이상 100% 미만인 변수 목록:")
print(f1_95_100_vars)

print("\n F1 = 90% 이상 100% 미만인 변수 목록:")
print(f1_90_100_vars)

print("\n F1 = 85% 이상 100% 미만인 변수 목록:")
print(f1_85_100_vars)

print("\n F1 = 80% 이상 100% 미만인 변수 목록:")
print(f1_80_100_vars)

print("\n F1 < 95%인 변수 목록:")
print(f1_below_95_vars)

print("\n F1 < 90%인 변수 목록:")
print(f1_below_90_vars)

print("\n F1 < 85%인 변수 목록:")
print(f1_below_85_vars)

print("\n F1 < 80%인 변수 목록:")
print(f1_below_80_vars)

F1-score가 정확히 100%인 변수 개수: 23
F1-score가 95% 이상 100% 미만인 변수 개수: 59
F1-score가 90% 이상 100% 미만인 변수 개수: 67
F1-score가 85% 이상 100% 미만인 변수 개수: 70
F1-score가 80% 이상 100% 미만인 변수 개수: 72
F1-score가 95% 미만인 변수 개수: 23
F1-score가 90% 미만인 변수 개수: 15
F1-score가 85% 미만인 변수 개수: 12
F1-score가 80% 미만인 변수 개수: 10

 F1 = 100%인 변수 목록:
['TIKM', 'SMTIK', 'UNITPITCH', 'BTBX', 'BTBY', 'TOLERANCEYM', 'TOLERANCEYP', 'STRIPX', 'STRIPY', 'PCPPANEL', 'ARNR', 'PLATEYIELD', 'ERDAT', 'BOMNR', 'MATNR', 'POSNR', 'PRLAYERTO', 'OHMTOLPM', 'MASWID', 'MASSPACE', 'AWWID', 'AWSPACE', 'PROCGR']

 F1 = 95% 이상 100% 미만인 변수 목록:
['TIK', 'TIKP', 'TIKRATE', 'DVPDATE', 'TARTIK', 'TIKDIFF', 'BALLBOT', 'BALLTOP', 'UNITSIZEX', 'UNITSIZEY', 'UNITPITCHY', 'SPLXOUTRATE', 'SPLXOUT', 'XOUTRATE', 'XOUT', 'TOLERANCEXM', 'TOLERANCEXP', 'UPNR', 'UPCOL', 'UPROW', 'PANELLOSSX', 'PANELLOSSY', 'ARM2NR', 'OKDAT', 'OKZET', 'REQNR', 'AEDAT', 'AEZET', 'LAYER', 'MATNRTIK', 'MATNRTIKT', 'TOLERANCEP', 'TOLERANCEM', 'LAYERTO', 'TIKAL', 'PROCSEQ', 'SVH', 'SVHPAD', 'SVH

### 범주형

In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

def evaluate_classification_targets(df, target_columns, test_size=0.2, random_state=42):
    results = []
    skipped = []

    for target_col in target_columns:
        print("=" * 80)
        print(f"🎯 분석 중인 타겟: {target_col}")
        df_copy = df.copy()

        feature_columns = [col for col in df_copy.columns if col not in [target_col, 'SPECNR']]

        # 결측치 처리
        for col in feature_columns:
            if df_copy[col].dtype == "object":
                df_copy[col] = df_copy[col].fillna("missing")
            else:
                df_copy[col] = df_copy[col].fillna(0)

        df_copy = df_copy.dropna(subset=[target_col])

        # 샘플 부족 시 스킵
        if df_copy.shape[0] < 10:
            reason = f"샘플 수 부족 ({df_copy.shape[0]}개)"
            print(f" ⚠️ 타겟 '{target_col}' 건너뜀 - {reason}")
            skipped.append({"Target": target_col, "Reason": reason})
            continue

        X = df_copy[feature_columns].copy()
        y = df_copy[target_col].copy().astype(str)

        # 라벨 인코딩
        y_le = LabelEncoder()
        y_encoded = y_le.fit_transform(y)

        # 피처 인코딩
        for col in feature_columns:
            if X[col].dtype == "object" or X[col].apply(type).nunique() > 1:
                le = LabelEncoder()
                X[col] = le.fit_transform(X[col].astype(str))

        class_counts = np.bincount(y_encoded)
        #print(f" 클래스 분포: {dict(zip(y_le.classes_, class_counts))}")

        # 단일 클래스인 경우 스킵
        if len(np.unique(y_encoded)) < 2:
            reason = "단일 클래스"
            print(f" ⚠️ 타겟 '{target_col}' 건너뜀 - {reason}")
            skipped.append({"Target": target_col, "Reason": reason})
            continue

        stratify_param = y_encoded if np.min(class_counts) >= 2 else None

        # 데이터 분할
        try:
            X_train, X_test, y_train, y_test = train_test_split(
                X, y_encoded, test_size=test_size, random_state=random_state,
                stratify=stratify_param
            )
        except Exception as e:
            reason = f"데이터 분할 실패: {str(e)}"
            print(f" ⚠️ 타겟 '{target_col}' 데이터 분할 실패: {reason}")
            skipped.append({"Target": target_col, "Reason": reason})
            continue

        models = {
            "XGBoost": XGBClassifier(
                use_label_encoder=False, eval_metric="logloss",
                random_state=random_state, verbosity=0
            ),
            "RandomForest": RandomForestClassifier(random_state=random_state)
        }

        # 모델 학습 및 평가
        for model_name, model in models.items():
            try:
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)
                f1_macro = f1_score(y_test, y_pred, average="macro")
                f1_weighted = f1_score(y_test, y_pred, average="weighted")

                print(f" ✅ {model_name} 결과 - Macro F1: {f1_macro:.4f}, Weighted F1: {f1_weighted:.4f}")

                results.append({
                    "Target": target_col,
                    "Model": model_name,
                    "Macro F1": round(f1_macro, 4),
                    "Weighted F1": round(f1_weighted, 4)
                })

            except Exception as e:
                reason = f"모델 학습 실패: {model_name} - {str(e)}"
                print(f" ⚠️ 타겟 '{target_col}' 모델 '{model_name}' 학습 실패: {reason}")
                skipped.append({"Target": target_col, "Reason": reason})

    result_df = pd.DataFrame(results)
    skipped_df = pd.DataFrame(skipped)

    # 각 타겟별 최고 성능 모델 선택
    if not result_df.empty:
        best_models = (
            result_df.sort_values(["Target", "Macro F1"], ascending=[True, False])
            .groupby("Target")
            .head(1)
            .reset_index(drop=True)
        )
        print("\n🏆 타겟별 최고 성능 모델 요약:")
        print(best_models)
    else:
        best_models = pd.DataFrame(columns=["Target", "Model", "Macro F1", "Weighted F1"])
        print("\n⚠️ 평가 가능한 타겟이 없습니다.")

    return result_df, skipped_df, best_models

In [13]:
target_columns = ['MANDT', 'DISKNR', 'SPECTYPE', 'CONMETHOD', 'DEFUALTTIK', 'SMTIKP', 'SMTIKM', 'DVPNO', 'SYS_TOOL', 'SYS_KDMAT', 'SYS_REV', 'CAM_KDMAT', 'CAM_REV', 'OLD_TOOL', 'CAM_TOOL', 'HZ', 'HZ_CHK', 'MATNR_INFO', 'IMAGE_SCALE', 'REAL', 'TEST', 'EWONHWA', 'TIKBASE', 'UNITTOLM', 'UNITTOLP', 'UNITPITCHM', 'UNITPITCHP', 'UNITPITCHMY', 'UNITPITCHPY', 'STRIPTOLM', 'STRIPTOLP', 'STRIPTOLM_Y', 'STRIPTOLP_Y', 'LPC', 'PKGCODE', 'SYMMETRY', 'KUNNRMATNR', 'SPLXOPT', 'XOPT', 'PURPOSE', 'MATNRGRADE', 'GRADEITEM', 'GRADECONTENT', 'ULNEED', 'ULPOS', 'ULPROC', 'ULFORMAT', 'WEEKNEED1', 'WEEKPOS1', 'WEEKPROC1', 'WEEKFORMAT1', 'WEEKNEED2', 'WEEKPOS2', 'WEEKPROC2', 'WEEKFORMAT2', 'PSRTENT', 'PLUGGING', 'WATERMK', 'DUALTOOL', 'EDMARK', 'FILTERTRA', 'PSRAPPR', 'TERM4', 'TWODBARCODE', 'UNIT2DID', 'HALOGENFREE', 'MEABM', 'GNSPECNR', 'UNITNO', 'STRIPNO', 'PRTYPE', 'MOLDGATE_ARRANGE', 'MOLDGATE_ARRTXT', 'STRIP_ARRANGE', 'STRIP_ARGTXT', 'STRCT', 'DETTYPE', 'LAMIN', 'LASER', 'M1', 'M2', 'M2_M1', 'B1', 'B2', 'B3', 'CAMYIELD', 'CONVERSION', 'SHIPPING', 'SSTRAY', 'NETPR', 'WAERK', 'YILD', 'SIZECHK', 'STICKY', 'LOWERLEFTX', 'LOWERLEFTXP', 'LOWERLEFTXM', 'LOWERLEFTY', 'LOWERLEFTYP', 'LOWERLEFTYM', 'PRECOATTOP', 'PRECOATBOT', 'SOPTOP', 'SOPBOT', 'ROTATE', 'LAYERNR', 'UPBLOCK', 'PANELMNLX', 'PANELMNLY', 'INTERVALX', 'INTERVALY', 'WORKSIZEX', 'WORKSIZEY', 'TRIMSIZEX01', 'TRIMSIZEX02', 'TRIMSIZEX03', 'TRIMSIZEX04', 'TRIMSIZEX05', 'TRIMSIZEX06', 'TRIMSIZEX07', 'TRIMSIZEX08', 'TRIMSIZEX09', 'TRIMSIZEX10', 'TRIMSIZEX11', 'TRIMSIZEX12', 'TRIMSIZEX13', 'TRIMSIZEX14', 'TRIMSIZEX15', 'TRIMSIZEX16', 'TRIMSIZEX17', 'TRIMSIZEX18', 'TRIMSIZEX19', 'TRIMSIZEX20', 'TRIMSIZEY01', 'TRIMSIZEY02', 'TRIMSIZEY03', 'TRIMSIZEY04', 'TRIMSIZEY05', 'TRIMSIZEY06', 'TRIMSIZEY07', 'TRIMSIZEY08', 'TRIMSIZEY09', 'TRIMSIZEY10', 'TRIMSIZEY11', 'TRIMSIZEY12', 'TRIMSIZEY13', 'TRIMSIZEY14', 'TRIMSIZEY15', 'TRIMSIZEY16', 'TRIMSIZEY17', 'TRIMSIZEY18', 'TRIMSIZEY19', 'TRIMSIZEY20', 'DBARCODE', 'SHEETX', 'SHEETY', 'PETCHBACK', 'SOP', 'SOSP', 'TAILESS', 'VIAFILL', 'CAPPLAT', 'VIP', 'VOP', 'CORELESS', 'CUDIRECT', 'HIPOT', 'IMPEDANCE', 'BPL', 'ZBC', 'CUBEAM', 'NIBEAM', 'PINMETHD', 'EO', 'MLCC', 'PRESS_FIT', 'FLAT_PAD', 'BTB', 'SMS', 'ETS', 'COPYCHK', 'STPNL', 'STACK3L', 'ETS_MTK', 'MIS', 'TOUCHWB', 'MCP', 'BUT_PLAT', 'SLC_PLAT', 'HAST', 'THIRDBVH', 'MAIN_REVS', 'MAIN_ITS', 'FULLITS', 'GLICAP', 'RVC', 'BBT', 'LAYERSTR1', 'LAYERSTR2', 'LAYERSTR3', 'SPECTYPE2', 'PRDHA', 'PRDHA_TXT', 'CONTIK', 'CONTIKP', 'CONTIKM', 'FPCTIK', 'FPCTIKP', 'FPCTIKM', 'INSTOOL', 'PKGUNIT', 'AS_START', 'AS_COMPLT', 'AS_HOLD', 'DR_START', 'DR_COMPLT', 'DR_HOLD', 'TOL_START', 'TOL_COMPLT', 'TOL_HOLD', 'MS_START', 'MS_COMPLT', 'MS_HOLD', 'APPROVED', 'OKNAM', 'SPECNRR', 'TOOLCOPY', 'JOBCASE', 'ACTIVE', 'LVORM', 'TOOLPART', 'KDGRP', 'TOOLNR', 'TOOLVER', 'TOOLYEAR', 'TOOLINFO', 'CHANGEPROC', 'CHANGELAYER', 'ORDERX', 'SPCTR', 'REASON', 'WATERMARK1', 'WATERMARK2', 'WATERMARK3', 'MIGRATION', 'INEMP', 'REQTY', 'COMMONTOOL', 'PSR_JEPGR_T', 'PSR_JEPGR_B', 'PSR_JIGGR_1', 'PSR_JIGGR_2', 'GAPCUPMAX', 'GAPCUPMIN', 'IDF', 'ODF', 'ERNAM', 'AENAM', 'ETL_DT', 'rn', 'LAYER2', 'RAWGR', 'MAKTX', 'MAKER', 'SPEC', 'SPEC2', 'ZSIZE', 'COPPERTIK', 'COPPERTIK2', 'COPPERTIKTUP', 'COPPERTIKTDW', 'REVERSE', 'NOISSUE', 'KMPMG', 'AUSME', 'MEINS', 'SSEQNR', 'LABOR', 'PSRPR', 'MATNR2', 'MAKTX2', 'SUB', 'PPG_RSEQNR', 'SPEC3', 'SPEC4', 'SPEC5', 'SPEC6', 'SPEC7', 'SPEC8', 'SPEC9', 'SPEC10', 'SPEC11', 'SPEC12', 'ZPROCSEQ', 'SCALEX', 'SCALEY', 'ZAREA01', 'ZAREA02', 'ZAREA03', 'ZAREA04', 'ZAREA05', 'ZAREA06', 'ZAREA07', 'ZAREA08', 'ZAREA09', 'ZAREA10', 'LAYERFR', 'TARDILONG', 'TARDISHORT', 'TRIMSIZEX', 'TRIMSIZEY', 'PINMETH', 'PINSTRC', 'CTOC', 'PROCCD', 'PROCNM', 'RSEQNR', 'PRLAYERFR', 'PRLAYER', 'NPRLAYER', 'STLAYER', 'TOOLS', 'STACK', 'CLASSES', 'DIRECTION', 'CLEARANCE', 'ENTRYBOARD', 'DEPTHSPEC', 'PRLAYERDEP', 'NPRLAYERDEP', 'STLAYERDEP', 'DPRLAYERDEP', 'BDRILLTYPE', 'DRILLCONM', 'DRILLCONM_CHK', 'SPCG', 'COTINGBIT', 'ZEPTHPLUS', 'ZEPTHMINUS', 'STUBMIN', 'STUBMAX', 'EXCE_SPEC', 'LASERTYPE_KEY', 'SHOT', 'PROCMETHOD', 'LASERTYPE', 'MIDDLE1', 'SVH2', 'SVH2PAD', 'SVH2AR', 'TOP2', 'MIDDLE2', 'BOTTOM2', 'CUETPROC', 'CUETSIZE', 'CUETSIZEMIN', 'CUETSIZEMAX', 'ZDEPTH', 'ZASPECTR', 'ZAREXCEPTION', 'LOCATION', 'THMAX', 'GND', 'MEASUREBA', 'EXPMD', 'EXPDIVC', 'UNITCURATE', 'KITCURATE', 'SIGMETHOD', 'CFTHICK', 'SPCCHK', 'LAYERPROCFR', 'LAYERPROCTO', 'DIMPLE', 'CUTMIN', 'CUTMAX', 'IVH1PAD', 'IVH2PAD', 'IVH3PAD', 'IVH4PAD', 'MAINPAD', 'LVH1PAD', 'LVH2PAD', 'LVH3PAD', 'LVH4PAD', 'LVH5PAD', 'LVH6PAD', 'LVH7PAD', 'LVH8PAD', 'IVH1PADMAX', 'IVH2PADMAX', 'IVH3PADMAX', 'IVH4PADMAX', 'MAINPADMAX', 'LVH1PADMAX', 'LVH2PADMAX', 'LVH3PADMAX', 'LVH4PADMAX', 'LVH5PADMAX', 'LVH6PADMAX', 'LVH7PADMAX', 'LVH8PADMAX', 'AOIGUBUN', 'G_SPACE', 'G_SPACE_MIN', 'G_SPACE_MAX', 'IN_G_SPACE', 'IN_G_SPACE_MIN', 'IN_G_SPACE_MAX', 'VIAPAD', 'VIAPADMIN', 'VIAPADMAX', 'PADTYPE', 'REF', 'MOLDGATE', 'BOT', 'LOCATION1', 'BGAFM', 'GNDTXT', 'ILEWID', 'ILEWIDMN', 'ILEWIDMX', 'OHM', 'BOGANG', 'ETA_MIN', 'IMPTYPE', 'CUPNO', 'ENGLOC', 'REFLAYER1', 'REFLAYER2', 'IMPTXT', 'HOLESEQ', 'LENTH', 'PTH', 'HOLEDIS', 'BRIEFS', 'VENTH', 'ORGSIZE', 'PROCGRNM', 'SEPCSTAT', 'LAYERPARDE', 'PROCMOD', 'REMARK', 'PRSEQNR', 'SUBGP', 'PREWORK', 'ZAUTO', 'STARGET', 'COMPALLOC', 'LAYERPARDEX', 'ROUTINENR', 'AOINR', 'SCALE510', 'SCALE415', 'TRIML', 'TRIMSH', 'DIVISION', 'TARGETDST', 'TRIMSHPL', 'TRIMSHMN', 'GUIDETYPE', 'JPINFTOP', 'JPINFBOT', 'FLEXCH', 'AUTOPUNCH', 'TARGETDSTXTOP', 'TARGETDSTXBOT', 'TARGETDSTYLEFT', 'TARGETDSTYRIGHT', 'JIG_CHK']  # 타겟 컬럼들
#result_df, skipped_df = evaluate_classification_targets(df, target_columns)
result_df, skipped_df, best_models = evaluate_classification_targets(df, target_columns)

🎯 분석 중인 타겟: MANDT
 ⚠️ 타겟 'MANDT' 건너뜀 - 단일 클래스
🎯 분석 중인 타겟: DISKNR
 ⚠️ 타겟 'DISKNR' 모델 'XGBoost' 학습 실패: 모델 학습 실패: XGBoost - Invalid classes inferred from unique values of `y`.  Expected: [   0    1    2 ... 2614 2615 2616], got [   0    2    3 ... 2933 2934 2935]
 ✅ RandomForest 결과 - Macro F1: 0.4330, Weighted F1: 0.6738
🎯 분석 중인 타겟: SPECTYPE
 ✅ XGBoost 결과 - Macro F1: 0.9914, Weighted F1: 0.9951
 ✅ RandomForest 결과 - Macro F1: 0.9891, Weighted F1: 0.9940
🎯 분석 중인 타겟: CONMETHOD
 ✅ XGBoost 결과 - Macro F1: 0.9769, Weighted F1: 0.9929
 ✅ RandomForest 결과 - Macro F1: 0.9804, Weighted F1: 0.9929
🎯 분석 중인 타겟: DEFUALTTIK
 ⚠️ 타겟 'DEFUALTTIK' 건너뜀 - 단일 클래스
🎯 분석 중인 타겟: SMTIKP
 ⚠️ 타겟 'SMTIKP' 모델 'XGBoost' 학습 실패: 모델 학습 실패: XGBoost - Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4 5 6 7 8], got [0 1 2 3 4 6 7 8 9]
 ✅ RandomForest 결과 - Macro F1: 0.8559, Weighted F1: 0.9986
🎯 분석 중인 타겟: SMTIKM
 ⚠️ 타겟 'SMTIKM' 모델 'XGBoost' 학습 실패: 모델 학습 실패: XGBoost - Invalid classes inferred from unique v

In [14]:
# 성공 타겟 수 
success_targets = set(result_df['Target'].unique())

# 완전 스킵된 타겟 수 
skipped_hard = skipped_df[skipped_df['Reason'].str.contains('샘플 수 부족|단일 클래스|데이터 분할 실패')]
skipped_hard_targets = set(skipped_hard['Target'].unique())

# 모델 학습 실패가 있었던 타겟 (하지만 다른 모델은 성공했을 수도)
fit_failed = skipped_df[skipped_df['Reason'].str.contains('모델 학습 실패')]
fit_failed_targets = set(fit_failed['Target'].unique())

print("타겟 총수:", len(set(target_columns)))
print("성공 타겟 수:", len(success_targets))
print("완전 스킵된 타겟 수:", len(skipped_hard_targets))

# 제거된 사유별 타겟 수
print("\n사유별 '타겟 기준' 집계:")
for reason_key in ['단일 클래스', '샘플 수 부족', '모델 학습 실패']:
    targets = set(skipped_df[skipped_df['Reason'].str.contains(reason_key)]['Target'])
    print(f"- {reason_key}: {len(targets)}개 타겟")

타겟 총수: 470
성공 타겟 수: 195
완전 스킵된 타겟 수: 275

사유별 '타겟 기준' 집계:
- 단일 클래스: 161개 타겟
- 샘플 수 부족: 114개 타겟
- 모델 학습 실패: 65개 타겟


In [15]:
# 타겟별 최고 성능만 뽑기 (모델별 중 가장 좋은 F1 기준)
best_scores = (
    result_df.groupby("Target")[["Macro F1", "Weighted F1"]]
    .max()
    .reset_index()
)

# 어떤 점수를 기준으로 구간을 나눌지 선택 (Weighted F1 기준 예시)
score_col = "Weighted F1"

# 각 구간별 분류
perfect     = best_scores[best_scores[score_col] == 1.0]
over95      = best_scores[(best_scores[score_col] >= 0.95) & (best_scores[score_col] < 1.0)]
over90      = best_scores[(best_scores[score_col] >= 0.90) & (best_scores[score_col] < 1.0)]
over85      = best_scores[(best_scores[score_col] >= 0.85) & (best_scores[score_col] < 1.0)]
over80      = best_scores[(best_scores[score_col] >= 0.80) & (best_scores[score_col] < 1.0)]
under95     = best_scores[best_scores[score_col] < 0.95]
under90     = best_scores[best_scores[score_col] < 0.90]
under85     = best_scores[best_scores[score_col] < 0.85]
under80     = best_scores[best_scores[score_col] < 0.80]

# 출력
print(" F1 = 100%:", len(perfect))
print(perfect["Target"].tolist())

print("\n 95% 이상 ~ 100% 미만:", len(over95))
print(over90["Target"].tolist())

print("\n 90% 이상 ~ 100% 미만:", len(over90))
print(over90["Target"].tolist())

print("\n 85% 이상 ~ 100% 미만:", len(over85))
print(over85["Target"].tolist())

print("\n 80% 이상 ~ 100% 미만:", len(over80))
print(over80["Target"].tolist())

print("\n 95% 미만 타겟:", len(under95))
print("\n 90% 미만 타겟:", len(under90))
print("\n 85% 미만 타겟:", len(under85))
print("\n 80% 미만 타겟:", len(under80))

 F1 = 100%: 61
['AUSME', 'CAMYIELD', 'CLEARANCE', 'EXPMD', 'FLEXCH', 'HOLEDIS', 'HZ', 'IMPTYPE', 'KMPMG', 'LAMIN', 'LASER', 'LASERTYPE', 'LASERTYPE_KEY', 'LAYER2', 'LOWERLEFTX', 'LOWERLEFTXM', 'LOWERLEFTXP', 'LOWERLEFTY', 'LOWERLEFTYM', 'LOWERLEFTYP', 'MEINS', 'MOLDGATE_ARRANGE', 'MOLDGATE_ARRTXT', 'MS_HOLD', 'PANELMNLY', 'PINMETH', 'PROCCD', 'PROCMETHOD', 'PROCNM', 'RAWGR', 'SHEETX', 'SHEETY', 'SPEC5', 'STRIPTOLM_Y', 'STRIPTOLP_Y', 'STRIP_ARGTXT', 'STRIP_ARRANGE', 'SVH2', 'SVH2AR', 'SVH2PAD', 'TARGETDSTXBOT', 'TARGETDSTXTOP', 'TARGETDSTYLEFT', 'TARGETDSTYRIGHT', 'TOOLNR', 'TOOLYEAR', 'TRIML', 'TRIMSH', 'TRIMSIZEX', 'TRIMSIZEX01', 'TRIMSIZEX02', 'TRIMSIZEX03', 'TRIMSIZEY', 'TRIMSIZEY01', 'TRIMSIZEY02', 'TRIMSIZEY03', 'WATERMARK1', 'WATERMARK2', 'WATERMARK3', 'WEEKPROC1', 'WORKSIZEX']

 95% 이상 ~ 100% 미만: 85
['AENAM', 'BOTTOM2', 'CONMETHOD', 'COPPERTIK', 'COPPERTIK2', 'CUETPROC', 'CUETSIZE', 'DBARCODE', 'DIRECTION', 'DR_COMPLT', 'DR_START', 'ERNAM', 'EXCE_SPEC', 'EXPDIVC', 'GNSPECNR', 'H

In [29]:
print("\n 95% 미만 타겟:", under95["Target"].tolist())


 95% 미만 타겟: ['BRIEFS', 'CAM_KDMAT', 'CAM_REV', 'CAM_TOOL', 'CONVERSION', 'CUPNO', 'DISKNR', 'DIVISION', 'DVPNO', 'ETL_DT', 'GNDTXT', 'GNSPECNR', 'GRADECONTENT', 'GRADEITEM', 'INEMP', 'KUNNRMATNR', 'LOCATION', 'MAKTX', 'MATNR_INFO', 'OKNAM', 'OLD_TOOL', 'PADTYPE', 'PKGCODE', 'PRECOATBOT', 'PRECOATTOP', 'PRSEQNR', 'PRTYPE', 'PSR_JEPGR_B', 'PSR_JEPGR_T', 'PSR_JIGGR_1', 'PSR_JIGGR_2', 'REASON', 'REMARK', 'SCALE415', 'SCALE510', 'SHOT', 'SOPTOP', 'SPEC', 'SPEC4', 'SPECNRR', 'STRIPNO', 'SYS_KDMAT', 'SYS_REV', 'SYS_TOOL', 'TOL_COMPLT', 'TOL_HOLD', 'TRIMSHPL', 'UNITNO', 'ZSIZE']


In [27]:
single_class_targets = skipped_df[skipped_df['Reason'].str.contains('단일 클래스')]['Target'].tolist()
print(len(single_class_targets))
print("단일 클래스 타겟:", single_class_targets)

161
단일 클래스 타겟: ['MANDT', 'DEFUALTTIK', 'REAL', 'TEST', 'SYMMETRY', 'WEEKNEED1', 'WEEKFORMAT1', 'PLUGGING', 'TERM4', 'UNIT2DID', 'MEABM', 'DETTYPE', 'M1', 'B1', 'B2', 'B3', 'NETPR', 'YILD', 'STICKY', 'ROTATE', 'TRIMSIZEX06', 'TRIMSIZEX07', 'TRIMSIZEX08', 'TRIMSIZEX09', 'TRIMSIZEX10', 'TRIMSIZEX11', 'TRIMSIZEX12', 'TRIMSIZEX13', 'TRIMSIZEX14', 'TRIMSIZEX15', 'TRIMSIZEX16', 'TRIMSIZEX17', 'TRIMSIZEX18', 'TRIMSIZEX19', 'TRIMSIZEX20', 'TRIMSIZEY06', 'TRIMSIZEY07', 'TRIMSIZEY08', 'TRIMSIZEY09', 'TRIMSIZEY10', 'TRIMSIZEY11', 'TRIMSIZEY12', 'TRIMSIZEY13', 'TRIMSIZEY14', 'TRIMSIZEY15', 'TRIMSIZEY16', 'TRIMSIZEY17', 'TRIMSIZEY18', 'TRIMSIZEY19', 'TRIMSIZEY20', 'PETCHBACK', 'SOP', 'SOSP', 'VIAFILL', 'CAPPLAT', 'VIP', 'CORELESS', 'CUDIRECT', 'IMPEDANCE', 'EO', 'BTB', 'ETS', 'COPYCHK', 'STACK3L', 'HAST', 'MAIN_ITS', 'FULLITS', 'BBT', 'LAYERSTR1', 'LAYERSTR2', 'LAYERSTR3', 'CONTIK', 'CONTIKP', 'CONTIKM', 'FPCTIK', 'FPCTIKP', 'FPCTIKM', 'PKGUNIT', 'APPROVED', 'ACTIVE', 'TOOLVER', 'CHANGEPROC', 'CHANG

In [31]:
insufficient_sample_targets = skipped_df[skipped_df['Reason'].str.contains('샘플 수 부족')]['Target'].tolist()
print(len(insufficient_sample_targets))
print("샘플 수 부족 타겟:", insufficient_sample_targets)

114
샘플 수 부족 타겟: ['HZ_CHK', 'IMAGE_SCALE', 'EWONHWA', 'PURPOSE', 'ULNEED', 'ULPOS', 'ULPROC', 'ULFORMAT', 'WEEKNEED2', 'WEEKPOS2', 'WEEKPROC2', 'WEEKFORMAT2', 'PSRTENT', 'WATERMK', 'DUALTOOL', 'EDMARK', 'FILTERTRA', 'PSRAPPR', 'TWODBARCODE', 'HALOGENFREE', 'M2', 'M2_M1', 'SSTRAY', 'WAERK', 'SIZECHK', 'SOPBOT', 'TAILESS', 'VOP', 'HIPOT', 'BPL', 'ZBC', 'CUBEAM', 'NIBEAM', 'PINMETHD', 'MLCC', 'PRESS_FIT', 'FLAT_PAD', 'SMS', 'ETS_MTK', 'MIS', 'TOUCHWB', 'MCP', 'BUT_PLAT', 'SLC_PLAT', 'THIRDBVH', 'MAIN_REVS', 'GLICAP', 'RVC', 'SPECTYPE2', 'INSTOOL', 'AS_START', 'AS_COMPLT', 'AS_HOLD', 'DR_HOLD', 'MS_START', 'TOOLCOPY', 'LVORM', 'TOOLPART', 'KDGRP', 'ORDERX', 'MIGRATION', 'REQTY', 'COMMONTOOL', 'REVERSE', 'NOISSUE', 'LABOR', 'MATNR2', 'MAKTX2', 'SUB', 'SPEC7', 'SPEC8', 'SPEC9', 'SPEC10', 'SPEC11', 'SPEC12', 'ZAREA01', 'ZAREA02', 'ZAREA03', 'ZAREA04', 'ZAREA05', 'ZAREA06', 'ZAREA07', 'ZAREA08', 'ZAREA09', 'ZAREA10', 'PINSTRC', 'ENTRYBOARD', 'BDRILLTYPE', 'DRILLCONM', 'DRILLCONM_CHK', 'ZAREXCEP